# CNN

**Reference:** https://github.com/vinayakumarr/Network-Intrusion-Detection

**Dataset:** http://kdd.ics.uci.edu/databases/kddcup99/kddcup99.html

In [3]:
import numpy as np
np.random.seed(1337)  # for reproducibility

import pandas as pd

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Dense, Dropout, MaxPooling1D, Flatten

from sklearn.preprocessing import Normalizer

ModuleNotFoundError: No module named 'tensorflow.keras'

In [ ]:
DATA = '../data/'
MODELS = '../models/'
TEST = 0.3

In [ ]:
traindata = pd.read_csv(DATA+'kddcup.data.corrected.csv', header=None)
traindata

,0,1,2,3,4,5,6,7,8,9,...,32,33,34,35,36,37,38,39,40,41
0,0,tcp,http,SF,215,45076,0,0,0,0,...,0,0.0,0.0,0.00,0.00,0.0,0.00,0.0,0.0,normal.
1,0,tcp,http,SF,162,4528,0,0,0,0,...,1,1.0,0.0,1.00,0.00,0.0,0.00,0.0,0.0,normal.
2,0,tcp,http,SF,236,1228,0,0,0,0,...,2,1.0,0.0,0.50,0.00,0.0,0.00,0.0,0.0,normal.
3,0,tcp,http,SF,233,2032,0,0,0,0,...,3,1.0,0.0,0.33,0.00,0.0,0.00,0.0,0.0,normal.
4,0,tcp,http,SF,239,486,0,0,0,0,...,4,1.0,0.0,0.25,0.00,0.0,0.00,0.0,0.0,normal.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4898426,0,tcp,http,SF,212,2288,0,0,0,0,...,255,1.0,0.0,0.33,0.05,0.0,0.01,0.0,0.0,normal.
4898427,0,tcp,http,SF,219,236,0,0,0,0,...,255,1.0,0.0,0.25,0.05,0.0,0.01,0.0,0.0,normal.
4898428,0,tcp,http,SF,218,3610,0,0,0,0,...,255,1.0,0.0,0.20,0.05,0.0,0.01,0.0,0.0,normal.
4898429,0,tcp,http,SF,219,1234,0,0,0,0,...,255,1.0,0.0,0.17,0.05,0.0,0.01,0.0,0.0,normal.


In [ ]:
from utils import preprocess

traindata, testdata=preprocess.preprocess(traindata)

In [ ]:
X = traindata.drop(columns=['Y'])
Y = traindata['Y']
C = testdata['Y']
T = testdata.drop(columns=['Y'])

In [ ]:
scaler = Normalizer().fit(X)
trainX = scaler.transform(X)

scaler = Normalizer().fit(T)
testT = scaler.transform(T)

C:\Users\hites\miniconda3\lib\site-packages\sklearn\utils\validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
C:\Users\hites\miniconda3\lib\site-packages\sklearn\utils\validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
C:\Users\hites\miniconda3\lib\site-packages\sklearn\utils\validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
C:\Users\hites\miniconda3\lib\site-packages\sklearn\utils\validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(


In [ ]:
y_train= pd.get_dummies(Y)
y_test= pd.get_dummies(C)

In [ ]:
y_train.columns

Index(['back.', 'buffer_overflow.', 'ftp_write.', 'guess_passwd.', 'imap.',
       'ipsweep.', 'land.', 'loadmodule.', 'multihop.', 'neptune.', 'nmap.',
       'normal.', 'perl.', 'phf.', 'pod.', 'portsweep.', 'rootkit.', 'satan.',
       'smurf.', 'spy.', 'teardrop.', 'warezclient.', 'warezmaster.'],
      dtype='object')

In [ ]:
# reshape input to be [samples, time steps, features]
X_train = np.reshape(trainX, (trainX.shape[0],trainX.shape[1],1))
X_test = np.reshape(testT, (testT.shape[0],testT.shape[1],1))

In [ ]:
model = Sequential()
model.add(Conv1D(8, 5, padding="same", activation="relu", input_shape=X_train.shape[1:]))
model.add(Conv1D(8, 5, padding="same", activation="relu"))
model.add(MaxPooling1D(2))
model.add(Flatten())
model.add(Dropout(0.1))
model.add(Dense(y_train.shape[1], activation="softmax"))

In [ ]:
# define optimizer and objective, compile model
model.compile(loss="categorical_crossentropy", optimizer="adam",metrics=['accuracy'])

In [ ]:
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 122, 8)            48        
                                                                 
 conv1d_1 (Conv1D)           (None, 122, 8)            328       
                                                                 
 max_pooling1d (MaxPooling1D  (None, 61, 8)            0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 488)               0         
                                                                 
 dropout (Dropout)           (None, 488)               0         
                                                                 
 dense (Dense)               (None, 23)                11247     
                                                        

In [ ]:
# train
model.fit(X_train, y_train, epochs=3, validation_split=0.4, shuffle=False, batch_size=128)

Epoch 1/3
16073/16073 [==============================] - 68s 4ms/step - loss: 0.1361 - accuracy: 0.9686 - val_loss: 0.0469 - val_accuracy: 0.9904
Epoch 2/3
16073/16073 [==============================] - 57s 4ms/step - loss: 0.0654 - accuracy: 0.9846 - val_loss: 0.0106 - val_accuracy: 0.9968
Epoch 3/3
16073/16073 [==============================] - 61s 4ms/step - loss: 0.0559 - accuracy: 0.9890 - val_loss: 0.0083 - val_accuracy: 0.9972


In [ ]:
model.evaluate(X_test, y_test)

InternalError: Failed copying input tensor from /job:localhost/replica:0/task:0/device:CPU:0 to /job:localhost/replica:0/task:0/device:GPU:0 in order to run _EagerConst: Dst tensor is not initialized.

In [ ]:
model.save(MODELS+"cnn_model.hdf5")